# Documentation for Using the Script to Build AndroidAAPS

This script is designed to help you build the AndroidAAPS application and save the built APK to your Google Drive. Please follow the instructions below to ensure a smooth process.

## 1. Running the Script

To run the script, click on the top menu bar and select `Runtime`, then click on `Run all`. The script will start executing, and you will be prompted to follow the necessary steps.

## 2. Google Account and Google Drive Connection

When you run this script, you will be prompted to log in with your Google Account. This is required for the script to access your Google Drive and save the built APK. Please follow the authentication steps to grant the necessary permissions for the script to connect to your Google Drive.

## 3. Default Save Locations

By default, the script will save the built APK in the following directory in your Google Drive: `AAPS/releases/`

The key will be saved at: `AAPS/key/key.jks`, and the key password will be saved at: `AAPS/key/key_password.txt`

## 4. Using an Existing Key

If you already have a key that you want to use for the build, you should place it at the following location in your Google Drive: `AAPS/key/key.jks`

Alternatively, you can modify the `KEYSTORE_FILE` variable in the script to point to the location of your existing key file.

Additionally, you will need to update the `KEY_PASSWORD` variable in the script with the password for your existing key.

Please ensure that you've placed the key file in the correct location and updated the necessary variables before running the script to avoid errors.

## 5. Changing the Build Branch

To build a different branch of the AndroidAAPS application, you can modify the value of the `AAPS_BRANCH` variable in the script. Set the variable to the desired branch name, and the script will build the application from that branch.

## 6. Confirming Terms and Conditions

Before the build process begins, you will be prompted to confirm that you have read and agreed to the Terms and Conditions. If you have read and agreed to the Terms and Conditions, type 'yes' and the build process will start.

**Please be patient, as the script will take approximately 15 to 20 minutes to complete.**

By following this documentation, you will be able to successfully build the AndroidAAPS application and save the APK to your Google Drive. If you encounter any issues or need further assistance, please reach out for help.

In [1]:
# Set AAPS_HOME variable to default value
AAPS_HOME = "/content/gdrive/MyDrive/AAPS"
AAPS_BRANCH = "master"
KEY_HOME = f"{AAPS_HOME}/key"
KEYSTORE_FILE = f"{KEY_HOME}/key.jks"
KEYSTORE_ALIAS = "key0"
KEY_PASSWORD = ""

# Terms and Conditions

## Disclaimer
This script is provided "as is" without warranty of any kind, either expressed or implied, including, but not limited to, the implied warranties of merchantability and fitness for a particular purpose. The entire risk as to the quality and performance of the script is with you. Should the script prove defective, you assume the cost of all necessary servicing, repair, or correction.

## Scope of Usage
This script is intended for educational and informational purposes only. It is designed to help users build the AndroidAAPS application themselves. By using this script, you agree and acknowledge that you are solely responsible for building the AndroidAAPS application and that the developer of this script is not distributing the built application.

## Limitation of Liability
In no event, unless required by applicable law or agreed to in writing, will the developer of this script be liable to you for damages, including any general, special, incidental, or consequential damages arising out of the use or inability to use the script (including but not limited to loss of data or data being rendered inaccurate or losses sustained by you or third parties or a failure of the script to operate with any other programs), even if the developer has been advised of the possibility of such damages.

## Acknowledgement of Risks
By using this script, you acknowledge and agree that you understand the risks associated with building and using the AndroidAAPS application. You assume full responsibility for any consequences that may arise from the use of the built application and agree to indemnify, defend, and hold harmless the developer of this script from any claims, damages, losses, or expenses resulting from your use of the application.

## Agreement to Additional Licenses
By using this script, you also agree to comply with the licenses of the AndroidAPS and the Android SDK.

For the AndroidAPS license, please refer to the following link: https://github.com/nightscout/AndroidAPS/blob/master/LICENSE.txt

For the Android SDK licenses, please refer to the following links:

* Android SDK Terms and Conditions: https://developer.android.com/studio/terms
* Google APIs Terms of Service: https://developers.google.com/terms/
* Google Play Terms of Service: https://play.google.com/about/play-terms/index.html

It is your responsibility to read, understand, and abide by the terms and conditions of these licenses while using this script and any associated software.

By using this script, you confirm that you have read, understood, and agree to be bound by these terms and conditions. If you do not agree to these terms, you should not use this script.

In [ ]:
def confirm_terms_and_conditions():
    print("Please carefully read and review the Terms and Conditions provided in the documentation.")
    print("By proceeding to use this script, you confirm that you have read, understood, and agreed to be bound by these terms and conditions.")
    response = input("Do you agree to the Terms and Conditions? Enter 'yes' to continue or 'no' to exit: ").lower()
    
    if response == 'yes':
        print("You have agreed to the Terms and Conditions. The script will now proceed.")
    else:
        print("You have not agreed to the Terms and Conditions. The script will now exit.")
        exit()

confirm_terms_and_conditions()

# Mount Google Drive folder to /content/gdrive
from google.colab import drive
drive.mount('/content/gdrive')

!mkdir -p $AAPS_HOME/releases
!mkdir -p $KEY_HOME

# Create a new key and password if not exists
import os.path
if not os.path.exists(KEYSTORE_FILE):
    # Generate new key and password
    import string
    import random
    chars = string.ascii_letters + string.digits
    KEY_PASSWORD = ''.join(random.choice(chars) for _ in range(16))
    print(f"New key password: {KEY_PASSWORD}")
    os.system(f"keytool -genkey -v -keystore {KEYSTORE_FILE} -alias {KEYSTORE_ALIAS} -storepass {KEY_PASSWORD} -keypass {KEY_PASSWORD} \
               -keyalg RSA -keysize 2048 -validity 10000 -dname 'CN=Unknown, OU=Unknown, O=Unknown, L=Unknown, ST=Unknown, C=Unknown'")
    # Save password to file
    with open(f"{KEY_HOME}/key_password.txt", 'w') as f:
        f.write(KEY_PASSWORD)
    # Prompt user to save key and password
    print("New key generated. Please save the key and password in a safe place.")

# Install dependencies and set up Android SDK
!apt-get update && \
  apt-get install -y git wget unzip libarchive-tools && \
  rm -rf /var/lib/apt/lists/*
  
!wget -q https://dl.google.com/android/repository/commandlinetools-linux-9477386_latest.zip -O android-sdk.zip && \
  mkdir -p /opt/android-sdk-linux/cmdline-tools/latest && \
  unzip -q android-sdk.zip -d /tmp && \
  mv /tmp/cmdline-tools/* /opt/android-sdk-linux/cmdline-tools/latest && \
  rm android-sdk.zip
  
!yes | /opt/android-sdk-linux/cmdline-tools/latest/bin/sdkmanager --licenses
!yes | /opt/android-sdk-linux/cmdline-tools/latest/bin/sdkmanager "platforms;android-28" "build-tools;28.0.3"

# Clone AndroidAPS repository to /src
!apt-get update && \
  apt-get install -y git && \
  rm -rf /var/lib/apt/lists/*
  
!git clone --branch $AAPS_BRANCH https://github.com/nightscout/AndroidAPS.git /src/AndroidAPS

ANDROID_HOME = "/opt/android-sdk-linux"
!echo "sdk.dir=$ANDROID_HOME" > /src/AndroidAPS/local.properties

# Fetch AAPS_VERSION
%cd /src/AndroidAPS
aaps_version = !grep -oE 'version "\S+"' app/build.gradle | cut -d'"' -f2
aaps_version = aaps_version[0]

# Define the output apk name using Python
if AAPS_BRANCH == "master":
    output_apk_name = f"AAPS_{aaps_version}.apk"
else:
    output_apk_name = f"AAPS_{AAPS_BRANCH}_{aaps_version}.apk"

# Build apk and save to mounted folder
!./gradlew assembleFullRelease --no-watch-fs \
  -Pandroid.injected.signing.store.file=$KEYSTORE_FILE \
  -Pandroid.injected.signing.store.password=$KEY_PASSWORD \
  -Pandroid.injected.signing.key.alias=$KEYSTORE_ALIAS \
  -Pandroid.injected.signing.key.password=$KEY_PASSWORD \
  && cp app/build/outputs/apk/full/release/app-full-release.apk $AAPS_HOME/releases/{output_apk_name}
